In [ ]:
!pip install qdrant-client openai

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.http import models

load_dotenv('/content/env.txt')

True

In [ ]:
#This code is to create the client.
client = QdrantClient(
    os.getenv("qdrant_host"),
    api_key=os.getenv("qdrant_api_key")
)

In [ ]:
# This section will config the collection
collection_config = models.VectorParams(
    size=1536,
    distance=models.Distance.COSINE
)

#This section will create a collection
client.create_collection(
    collection_name = os.getenv('qdrant_collection_name'),
    vectors_config = collection_config
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `my_collection` already exists!"},"time":0.020904401}'

In [ ]:
print(client.get_collections())

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def generate_bert_embedding(text):
    # Tokenize and encode the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Pass through the BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the CLS token's embedding as the sentence embedding
    embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding.numpy().tolist()

text = "I want to buy a new laptop."
print(generate_bert_embedding(text))


OSError: johngiorgi/biobert_v1.1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from qdrant_client.http.models import VectorParams

# Step 3: Initialize Qdrant client
qdrant_client = QdrantClient(url="https://bf6ac2ef-4068-4270-b2d9-41ce82519941.us-east4-0.gcp.cloud.qdrant.io:6333",
                             api_key="bYvc5aJwP1xWI1AEM6G-V5NcKfkdsbmlBcLTipKUnx5tCCanYAQLDg")  # Replace with your Qdrant URL

# Step 4: Create a collection in Qdrant
collection_name = "bert_collection"
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=768,          # Match the vector size of the BERT model
        distance="Cosine"  # Metric for similarity search
    )
)
print("Collection created successfully.")

<ipython-input-22-bc0e0858650c>:9: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Collection created successfully.


In [ ]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        {
            "id": 1,
            "vector": vector,
            "payload": {"text": text}  # Optional metadata
        }
    ]
)
print("BERT embedding inserted into Qdrant.")

BERT embedding inserted into Qdrant.


In [ ]:
# Generate a query vector using BERT
query_text = "I want to buy a new laptop."
query_vector = generate_bert_embedding(query_text)

# Search in Qdrant
search_results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3  # Number of results to return
)

# Display results
for result in search_results:
    print(f"ID: {result.id}, Payload: {result.payload}, Score: {result.score}")


ID: 1, Payload: {'text': 'I want to buy a new laptop.'}, Score: 0.7754436
